In [1]:
import numpy as np 
import pandas as pd


In [56]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [57]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [58]:
df = pd.merge(df,movie_titles,on='item_id')                  # merging datasets
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [59]:
df.shape

(100003, 5)

In [60]:
df.groupby('title')['rating'].mean().sort_values(ascending=False).head()

title
Marlene Dietrich: Shadow and Light (1996)     5.0
Prefontaine (1997)                            5.0
Santa with Muscles (1996)                     5.0
Star Kid (1997)                               5.0
Someone Else's America (1995)                 5.0
Name: rating, dtype: float64

In [61]:
df['title'].value_counts().reset_index()['title'].mean()

60.09795673076923

In [63]:
x=df['title'].value_counts()>70
x[x]

Star Wars (1977)                                   True
Contact (1997)                                     True
Fargo (1996)                                       True
Return of the Jedi (1983)                          True
Liar Liar (1997)                                   True
English Patient, The (1996)                        True
Scream (1996)                                      True
Toy Story (1995)                                   True
Air Force One (1997)                               True
Independence Day (ID4) (1996)                      True
Raiders of the Lost Ark (1981)                     True
Godfather, The (1972)                              True
Pulp Fiction (1994)                                True
Twelve Monkeys (1995)                              True
Silence of the Lambs, The (1991)                   True
Jerry Maguire (1996)                               True
Chasing Amy (1997)                                 True
Rock, The (1996)                                

In [65]:
y=x[x].index
y

Index(['Star Wars (1977)', 'Contact (1997)', 'Fargo (1996)',
       'Return of the Jedi (1983)', 'Liar Liar (1997)',
       'English Patient, The (1996)', 'Scream (1996)', 'Toy Story (1995)',
       'Air Force One (1997)', 'Independence Day (ID4) (1996)',
       ...
       'Othello (1995)', 'Seventh Seal, The (Sjunde inseglet, Det) (1957)',
       'Wedding Singer, The (1998)', 'Third Man, The (1949)',
       'Angels and Insects (1995)',
       'Things to Do in Denver when You're Dead (1995)', 'Restoration (1995)',
       'Secret of Roan Inish, The (1994)',
       'How to Make an American Quilt (1995)', 'Diabolique (1996)'],
      dtype='object', length=462)

In [68]:
new_list=df[df['title'].isin(y)]

In [96]:
df['user_id'].value_counts().reset_index()['user_id'].mean()

105.93538135593221

In [110]:
a=df['user_id'].value_counts()>100
b=a[a].index

In [99]:
new_list=new_list[new_list['user_id'].isin(b)]

In [100]:
movie_pivot=new_list.pivot_table(columns='user_id',index='title',values='rating')

In [101]:
movie_pivot.fillna(0,inplace=True)

In [102]:
from scipy.sparse import csr_matrix
movie_sparse=csr_matrix(movie_pivot)

In [103]:
from sklearn.neighbors import NearestNeighbors      
model=NearestNeighbors(algorithm='brute')

In [104]:
model.fit(movie_sparse)    # training the system

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [105]:
# writing a function which takes in movie names and recommends similar movies


def movie_recommend(movie_name):
    movie_id=np.where(movie_pivot.index==movie_name)[0][0]
    distances, suggestions=model.kneighbors(movie_pivot.iloc[movie_id,:].values.reshape(1,-1),n_neighbors=6)
    
    for i in range(len(suggestions)):
        print(movie_pivot.index[suggestions[i]])

In [112]:
movie_recommend('Godfather, The (1972)')             # checking the code

Index(['Godfather, The (1972)', 'Godfather: Part II, The (1974)',
       'GoodFellas (1990)', 'Fargo (1996)', 'Taxi Driver (1976)',
       'Apocalypse Now (1979)'],
      dtype='object', name='title')
